In [6]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import os
import zipfile
import rasterio
from rasterio.merge import merge
from tqdm import tqdm

C:\Users\Ondrej\miniconda3\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\Ondrej\miniconda3\lib\site-packages\osmnx\projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/

# Street data

### Netherlands

In [3]:
region = "Netherlands"
G = ox.graph_from_place(region, network_type="all") # get all networks for the given region
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G) # convert the graph to a GeoDataFrame

# reproject to the matching projection with dtm and dsm
gdf_nodes = gdf_nodes.to_crs('EPSG:28992')
gdf_edges = gdf_edges.to_crs('EPSG:28992')

### City level

In [2]:
cities = ["Amsterdam, Netherlands", "Rotterdam, Netherlands", "Utrecht, Netherlands", "The Hague, Netherlands"]

all_gdf_nodes = []
all_gdf_edges = []

# Iterate over cities with progress tracking
total_cities = len(cities)
for i, city in enumerate(cities):
    G = ox.graph_from_place(city, network_type="all")
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

    gdf_nodes = gdf_nodes.to_crs('EPSG:28992') # reproject to the matching projection with dtm and dsm
    gdf_edges = gdf_edges.to_crs('EPSG:28992')

    all_gdf_nodes.append(gdf_nodes)
    all_gdf_edges.append(gdf_edges)
    # progress
    progress = f"{i+1}/{total_cities}"
    print(f"Processing city {city} ({progress})")

# merge into a geodataframe
merged_gdf_nodes = gpd.GeoDataFrame(pd.concat(all_gdf_nodes, ignore_index=True))
merged_gdf_edges = gpd.GeoDataFrame(pd.concat(all_gdf_edges, ignore_index=True))

In [22]:
# filter down to only geometries
edges_geometry = merged_gdf_edges[["geometry"]]

# save the file as gpkg
edges_output_file = 'merged_edges.gpkg'
edges_geometry.to_file(edges_output_file, driver='GPKG')

# DSM/DTM files from AHN3

The below download DSM/DTM tiles and merges them into a single .tif file. The input url links for the input text file can be found from the following website:https://app.pdok.nl/rws/ahn3/download-page/#, where tile url links look as follows: https://download.pdok.nl/rws/ahn3/v1_0/05m_dsm/R_02EZ1.ZIP

In [4]:
# download the tiles
with open('C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_list_dtm.txt', 'r') as file:
    tile_urls = file.read().splitlines()

output_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_dtm'
os.makedirs(output_directory, exist_ok=True)

# progress 
total_tiles = len(tile_urls)
downloaded_tiles = 0

for url in tile_urls:
    file_name = url.split('/')[-1]
    output_path = os.path.join(output_directory, file_name)
    response = requests.get(url)
    with open(output_path, 'wb') as f:
        f.write(response.content)
    print("Downloaded", file_name)
    print("Saved to", os.path.abspath(output_path))
    
    # update the progress
    downloaded_tiles += 1
    print("Progress: {}/{}".format(downloaded_tiles, total_tiles))

Downloaded M5_02EZ1.ZIP
Saved to C:\Users\Ondrej\2.5D-GreenViewIndex-Netherlands\Data collection\tiles_dtm\M5_02EZ1.ZIP
Progress: 1/1371
Downloaded M5_02EZ2.ZIP
Saved to C:\Users\Ondrej\2.5D-GreenViewIndex-Netherlands\Data collection\tiles_dtm\M5_02EZ2.ZIP
Progress: 2/1371
Downloaded M5_02FZ1.ZIP
Saved to C:\Users\Ondrej\2.5D-GreenViewIndex-Netherlands\Data collection\tiles_dtm\M5_02FZ1.ZIP
Progress: 3/1371
Downloaded M5_02FZ2.ZIP
Saved to C:\Users\Ondrej\2.5D-GreenViewIndex-Netherlands\Data collection\tiles_dtm\M5_02FZ2.ZIP
Progress: 4/1371
Downloaded M5_01GN1.ZIP
Saved to C:\Users\Ondrej\2.5D-GreenViewIndex-Netherlands\Data collection\tiles_dtm\M5_01GN1.ZIP
Progress: 5/1371
Downloaded M5_03DZ1.ZIP
Saved to C:\Users\Ondrej\2.5D-GreenViewIndex-Netherlands\Data collection\tiles_dtm\M5_03DZ1.ZIP
Progress: 6/1371
Downloaded M5_01GN2.ZIP
Saved to C:\Users\Ondrej\2.5D-GreenViewIndex-Netherlands\Data collection\tiles_dtm\M5_01GN2.ZIP
Progress: 7/1371
Downloaded M5_01HN1.ZIP
Saved to C:\Users

In [10]:
working_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/tiles_dtm'
output_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/extracted'

os.makedirs(output_directory, exist_ok=True)  # Create the 'extracted' directory

for file in os.listdir(working_directory):
    if zipfile.is_zipfile(os.path.join(working_directory, file)):
        with zipfile.ZipFile(os.path.join(working_directory, file)) as item:
            item.extractall(output_directory)
            print("Extracted files from:", file)

print("Files extracted and saved to:", output_directory)

Extracted files from: M5_01CZ1.ZIP
Extracted files from: M5_01CZ2.ZIP
Extracted files from: M5_01DZ1.ZIP
Extracted files from: M5_01DZ2.ZIP
Extracted files from: M5_01GN1.ZIP
Extracted files from: M5_01GN2.ZIP
Extracted files from: M5_01GZ1.ZIP
Extracted files from: M5_01GZ2.ZIP
Extracted files from: M5_01HN1.ZIP
Extracted files from: M5_01HN2.ZIP
Extracted files from: M5_01HZ1.ZIP
Extracted files from: M5_01HZ2.ZIP
Extracted files from: M5_02CN1.ZIP
Extracted files from: M5_02CN2.ZIP
Extracted files from: M5_02CZ1.ZIP
Extracted files from: M5_02CZ2.ZIP
Extracted files from: M5_02DN1.ZIP
Extracted files from: M5_02DZ1.ZIP
Extracted files from: M5_02DZ2.ZIP
Extracted files from: M5_02EZ1.ZIP
Extracted files from: M5_02EZ2.ZIP
Extracted files from: M5_02FZ1.ZIP
Extracted files from: M5_02FZ2.ZIP
Extracted files from: M5_02GN1.ZIP
Extracted files from: M5_02GN2.ZIP
Extracted files from: M5_02GZ1.ZIP
Extracted files from: M5_02GZ2.ZIP
Extracted files from: M5_02HN1.ZIP
Extracted files from

In [11]:
extracted_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection/extracted'
output_directory = 'C:/Users/Ondrej/2.5D-GreenViewIndex-Netherlands/Data collection'
output_file = os.path.join(output_directory, 'merged.tif')

os.makedirs(output_directory, exist_ok=True) 

# get all the tiles into a list
tif_files = []
for root, dirs, files in os.walk(extracted_directory):
    tif_files += [os.path.join(root, file) for file in files if file.endswith('.TIF')]

# total
total_files = len(tif_files)
print(total_files)

# merge the files
merged_arr, out_trans = merge(tif_files)

# output and save the file 
with rasterio.open(tif_files[0]) as src:
    meta = src.meta

meta.update({"driver": "GTiff",
             "height": merged_arr.shape[1],
             "width": merged_arr.shape[2],
             "transform": out_trans})

with rasterio.open(output_file, "w", **meta) as dst:
    dst.write(merged_arr)

1371
